In [1]:
# Cell 1: Path & imports

import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

DATA_ROOT = "../mvtec_anomaly_detection"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
# Cell 2: Dataset class

class MVTecDataset(torch.utils.data.Dataset):
    def __init__(self, root, product, split="train"):
        self.samples = []
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

        base = os.path.join(root, product, split)

        if split == "train":
            img_dir = os.path.join(base, "good")
            for f in os.listdir(img_dir):
                self.samples.append((os.path.join(img_dir, f), 0))
        else:
            for cls in os.listdir(base):
                label = 0 if cls == "good" else 1
                img_dir = os.path.join(base, cls)
                for f in os.listdir(img_dir):
                    self.samples.append((os.path.join(img_dir, f), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        return self.transform(img), label


In [3]:
# Cell 3: Select product & dataloader

PRODUCT = "bottle"   # đổi product ở đây
BATCH_SIZE = 16

train_ds = MVTecDataset(DATA_ROOT, PRODUCT, split="train")
test_ds  = MVTecDataset(DATA_ROOT, PRODUCT, split="test")

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

print("Train images:", len(train_ds))
print("Test images:", len(test_ds))


Train images: 209
Test images: 83


In [4]:
# Cell 4: CNN feature extractor

from torchvision.models import resnet18

class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = resnet18(pretrained=True)
        self.features = nn.Sequential(*list(backbone.children())[:-2])

    def forward(self, x):
        f = self.features(x)
        return f.mean(dim=[2, 3])  # Global Average Pooling

model = FeatureExtractor().to(DEVICE)
model.eval()


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:01<00:00, 41.8MB/s]


FeatureExtractor(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [5]:
# Cell 5: Feature extraction

def extract_features(model, loader):
    feats = []
    labels = []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE)
            f = model(x)
            feats.append(f.cpu())
            labels.extend(y.numpy())

    return torch.cat(feats), np.array(labels)


train_feats, _ = extract_features(model, train_loader)
test_feats, test_labels = extract_features(model, test_loader)

print("Train features:", train_feats.shape)
print("Test features:", test_feats.shape)


Train features: torch.Size([209, 512])
Test features: torch.Size([83, 512])


In [7]:
# Anomaly score = distance to nearest normal feature

from torch import cdist

scores = []
for f in test_feats:
    dist = cdist(f.unsqueeze(0), train_feats)
    scores.append(dist.min().item())

scores = np.array(scores)


In [8]:
# Cell 6: Quick sanity check

print("Scores (first 10):", scores[:10])
print("Labels (first 10):", test_labels[:10])

print("Mean score normal:", scores[test_labels == 0].mean())
print("Mean score defect:", scores[test_labels == 1].mean())


Scores (first 10): [ 8.05090523  7.84604502  9.84459972 11.64801025  5.95104837  4.72312641
  9.28154278  9.35680485  9.64975643  5.57543802]
Labels (first 10): [1 1 1 1 1 1 1 1 1 1]
Mean score normal: 3.027949333190918
Mean score defect: 7.077017874944778
